In [25]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import numpy as np
import os
from scipy.signal import argrelextrema
from detecta import detect_peaks

In [26]:
df_snhn=pd.read_csv("results_sheet/song_hn.csv")
df_snhn=df_snhn.sort_values(by=["compression"])

### BEzier on actual values

In [175]:
df_bezier_on_actl_Wts=pd.read_csv("results_sheet/04bezier_on_actl_Wts17Jan2021.csv")
df_bezier_on_actl_Wts_new=pd.read_csv("results_sheet/04bezier_on_actl_Wts18Jan2021.csv")
df_bezier_on_actl_Wts=pd.concat([df_bezier_on_actl_Wts,df_bezier_on_actl_Wts_new])
print(df_bezier_on_actl_Wts.shape)

(20024, 6)


In [176]:
df_bezier_on_actl_Wts.head()

,range_params,upscale,upscale_method,epoch_number,compression,val_top_1
0,"[1, 0.14, 0.14, 0.1]",0.24,+,0,0.783916,0.228516
1,"[1, 0.14, 0.14, 0.1]",0.24,+,1,0.853457,0.320215
2,"[1, 0.14, 0.14, 0.1]",0.24,+,2,0.891742,0.385840
3,"[1, 0.14, 0.14, 0.1]",0.24,+,3,0.914196,0.420020
4,"[1, 0.14, 0.14, 0.1]",0.24,+,4,0.927670,0.437402


In [177]:
df_bezier_on_actl_Wts_filtered=df_bezier_on_actl_Wts[(df_bezier_on_actl_Wts["range_params"]=="[0.9995, 0.9995, 0.9995, 0.9994]")
                                                    & (df_bezier_on_actl_Wts["upscale"]==0.05)]
print(df_bezier_on_actl_Wts_filtered.shape)

(1, 6)


In [207]:
fig = go.Figure()

df_bezier_on_actl_Wts=pd.read_csv("results_sheet/04bezier_on_actl_Wts17Jan2021.csv")
df_bezier_on_actl_Wts_new=pd.read_csv("results_sheet/04bezier_on_actl_Wts18Jan2021.csv")
df_bezier_on_actl_Wts=pd.concat([df_bezier_on_actl_Wts,df_bezier_on_actl_Wts_new])
df_bezier_on_actl_Wts=df_bezier_on_actl_Wts.sort_values(by=["upscale","epoch_number"],ascending=True)



df_bezier_on_actl_Wts=df_bezier_on_actl_Wts.sort_values(by=["range_params","upscale","upscale_method"],ascending=True)

for range_params in list(df_bezier_on_actl_Wts["range_params"].unique()):

    df_bezier_on_actl_Wts_range_params=df_bezier_on_actl_Wts[df_bezier_on_actl_Wts["range_params"]==range_params]
    for upscale_val in list(df_bezier_on_actl_Wts_range_params["upscale"].unique()):        
        df_bezier_on_actl_Wts_range_params_upscale=df_bezier_on_actl_Wts_range_params[df_bezier_on_actl_Wts_range_params["upscale"]==upscale_val]
        
        for upscale_method in list(df_bezier_on_actl_Wts_range_params_upscale["upscale_method"].unique()):
            if upscale_method=="+":
                continue
            df_bezier_on_actl_Wts_range_params_upscale_upmethod=df_bezier_on_actl_Wts_range_params_upscale[df_bezier_on_actl_Wts_range_params_upscale["upscale_method"]==upscale_method]
            df_bezier_on_actl_Wts_range_params_upscale_upmethod=df_bezier_on_actl_Wts_range_params_upscale_upmethod.sort_values(by=["epoch_number"],ascending=True)
#             if df_bezier_on_actl_Wts_range_params_upscale_upmethod.shape[0]<1500:
#                 continue
            ys=df_bezier_on_actl_Wts_range_params_upscale_upmethod["val_top_1"].values
            xs=df_bezier_on_actl_Wts_range_params_upscale_upmethod["compression"].values

            # print(xs,ys)
            fig.add_trace(go.Scatter(x=xs, y=ys,
                            mode='lines+markers',
                            name=str(upscale_method)+str(range_params)+str(upscale_val)))




xs=df_snhn["compression"]
ys=df_snhn["top_1"]
fig.add_trace(go.Scatter(x=xs, y=ys,
                mode='lines+markers',
                name="SongHan"))



fig.show()

### More involved method of getting better values

Compression more than song han and accuracy better than the same



In [197]:
# df_snhn=pd.read_csv("results_sheet/song_hn.csv")
df_snhn=pd.read_csv("results_sheet/song_hn.csv")
df_snhn=df_snhn.sort_values(by=["compression"])

df_bezier_on_actl_Wts=pd.read_csv("results_sheet/04bezier_on_actl_Wts17Jan2021.csv")
df_bezier_on_actl_Wts_new=pd.read_csv("results_sheet/04bezier_on_actl_Wts18Jan2021.csv")
df_bezier_on_actl_Wts=pd.concat([df_bezier_on_actl_Wts,df_bezier_on_actl_Wts_new])





snhnComp=list(df_snhn.compression)
snhnAcc=list(df_snhn.top_1)


best_compression_acc={}
best_compression_acc["range_params"]=[]
best_compression_acc["compression"]=[]
best_compression_acc["val_top_1"]=[]
best_compression_acc["upscale"]=[]
best_compression_acc["upscale_method"]=[]

best_compression_acc["epoch_number"]=[]

for i in range(len(snhnComp)-1):
#     print(snhnComp[i],snhnComp[i+1])
    acc_snhn=snhnAcc[i]
    # find all data in bezierOnMean whose compression lies between given compression
    df_bezier_on_actl_Wts_in_between=df_bezier_on_actl_Wts[(df_bezier_on_actl_Wts["compression"]>snhnComp[i])&(df_bezier_on_actl_Wts["compression"]<=snhnComp[i+1])]
    # in this range, save the one with the maximum accuracy
    if df_bezier_on_actl_Wts_in_between.shape[0]>=1:
        # data exists, lets find the best
        df_bezier_on_actl_Wts_in_between=df_bezier_on_actl_Wts_in_between.sort_values(by=["val_top_1"],ascending=True)
        bez_acc=list(df_bezier_on_actl_Wts_in_between["val_top_1"])[-1]
        if bez_acc>=acc_snhn:
            best_compression_acc["range_params"].append(list(df_bezier_on_actl_Wts_in_between["range_params"])[-1])
            best_compression_acc["compression"].append(list(df_bezier_on_actl_Wts_in_between["compression"])[-1])
            best_compression_acc["val_top_1"].append(list(df_bezier_on_actl_Wts_in_between["val_top_1"])[-1])
            best_compression_acc["upscale"].append(list(df_bezier_on_actl_Wts_in_between["upscale"])[-1])
            best_compression_acc["upscale_method"].append(list(df_bezier_on_actl_Wts_in_between["upscale_method"])[-1])
            best_compression_acc["epoch_number"].append(list(df_bezier_on_actl_Wts_in_between["epoch_number"])[-1])

            
            


df_bezier_on_actl_Wts_in_between_best=pd.DataFrame(best_compression_acc)
print(df_bezier_on_actl_Wts_in_between_best)            



                        range_params  compression  val_top_1  upscale  \
0               [1, 0.14, 0.14, 0.1]     0.963009   0.860645    0.200   
1               [1, 0.14, 0.14, 0.1]     0.966366   0.862402    0.200   
2               [1, 0.14, 0.14, 0.1]     0.968076   0.866699    0.200   
3               [1, 0.14, 0.14, 0.1]     0.970506   0.873145    0.200   
4               [1, 0.14, 0.14, 0.1]     0.971132   0.874219    0.200   
5               [1, 0.14, 0.14, 0.1]     0.974082   0.874609    0.200   
6               [1, 0.14, 0.14, 0.1]     0.977756   0.861523    0.240   
7               [1, 0.14, 0.14, 0.1]     0.978102   0.861719    0.240   
8               [1, 0.14, 0.14, 0.1]     0.979758   0.858594    0.280   
9               [1, 0.14, 0.14, 0.1]     0.980924   0.856250    0.320   
10              [1, 0.14, 0.14, 0.1]     0.982538   0.845703    0.400   
11  [0.9995, 0.9995, 0.9995, 0.9994]     0.989512   0.702148    0.220   
12  [0.9995, 0.9995, 0.9995, 0.9994]     0.989858  

In [198]:
fig = go.Figure()



xs=df_bezier_on_actl_Wts_in_between_best["compression"]
ys=df_bezier_on_actl_Wts_in_between_best["val_top_1"]
fig.add_trace(go.Scatter(x=xs, y=ys,
                mode='lines+markers',
                name="BezActuals"))




xs=df_snhn["compression"]
ys=df_snhn["top_1"]
fig.add_trace(go.Scatter(x=xs, y=ys,
                mode='lines+markers',
                name="SongHan"))



fig.show()

In [173]:
times=[1,1,1,5,10,20,30]
upsc_vals=[0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.01]
all_processes=[]
for i in range(len(times)):
    this_process=[upsc_vals[i] for j in range(times[i])]
    all_processes.extend(this_process)

print(all_processes)

[0.5, 0.4, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [11]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import pickle
import os
from scipy import stats
    

import math
import random
import pandas as pd


In [12]:
def nonzero(tensor):
    """Returns absolute number of values different from 0

    Arguments:
        tensor {numpy.ndarray} -- Array to compute over

    Returns:
        int -- Number of nonzero elements
    """
    return np.sum(tensor != 0.0)


def model_size(model, as_bits=False):
    """Returns absolute and nonzero model size

    Arguments:
        model {torch.nn.Module} -- Network to compute model size over

    Keyword Arguments:
        as_bits {bool} -- Whether to account for the size of dtype

    Returns:
        int -- Total number of weight & bias params
        int -- Out total_params exactly how many are nonzero
    """

    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = nonzero(tensor.detach().cpu().numpy())
        if as_bits:
            bits = dtype2bits[tensor.dtype]
            t *= bits
            nz *= bits
        total_params += t
        nonzero_params += nz
    return int(total_params), int(nonzero_params)

In [13]:
# LENET 300-100 for MNIST and comparison
class FashionMnistNet(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self):
        super(FashionMnistNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)        
        self.fc4 = nn.Linear(64, 10)
        self.fc4.is_classifier = True
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))        
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        top_1, top_5 = correct(out, labels,topk=(1,5))
#         print("Batch is ",batch[1].shape)
        
        top_1=top_1/batch[1].shape[0]
        top_5=top_5/batch[1].shape[0]

#         print("corr",top_1,top_5)
#         return {'val_loss': loss, 'val_acc': acc}
        return {'val_loss': loss, 'val_acc': acc, 'top_1': top_1, 'top_5': top_5}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        
        batch_top_1s = [x['top_1'] for x in outputs]
#         print(batch_top_1s)
        epoch_top_1 = torch.stack(batch_top_1s).mean()      # Combine top_1
        
        batch_top_5s = [x['top_5'] for x in outputs]
        epoch_top_5 = torch.stack(batch_top_5s).mean()      # Combine top_5
        
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(),
               'val_top_1': epoch_top_1.item(), 'val_top_5': epoch_top_5.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}, val_top_1: {:.4f}, val_top_5: {:.4f}".format(
                                epoch, result['val_loss'], result['val_acc'], 
                                result['val_top_1'], result['val_top_5']))
        
        
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


In [14]:
model=FashionMnistNet()
if torch.cuda.is_available():
    model=model.cuda()


In [15]:
total_size,nz_size=model_size(model)
compression=(total_size-nz_size)/total_size
print("Compression=",compression)


Compression= 0.0


In [23]:
model_state_path="model_state/04_bez_on_actuals/[1, 0.14, 0.14, 0.1]/0.24/+/190.pt"

In [24]:
if torch.cuda.is_available():
    model.load_state_dict(torch.load(model_state_path))
else:
    model.load_state_dict(torch.load(model_state_path,map_location=torch.device('cpu')))


In [22]:
total_size,nz_size=model_size(model)
compression=(total_size-nz_size)/total_size
print("Compression=",compression)


Compression= 0.9658060157685305
